In [1]:
import re

from itertools import cycle, combinations, permutations, tee
from collections import Counter, defaultdict, deque
from io import StringIO

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def read_input(day, fn=str.strip):
    """//
    Return a list of the input lines mapped by fn
    
    example: 
    >>> read_input('01', int)  # read input file, map all lines to int
    
    Inspired by Peter Norvig: https://github.com/norvig/pytudes
    
    """
    return list(map(fn, open(f'input\{day}.txt')))

def all_integers(s):
    """return all integers from a string"""
    return tuple(map(int, re.findall(r'-?\d+', s)))

# Day 16

In [2]:
testcase = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

In [3]:
test_list = [line.rstrip('\n') for line in testcase.split('\n\n')]
test_list[:3],test_list[-1], len(test_list)

(['class: 1-3 or 5-7\nrow: 6-11 or 33-44\nseat: 13-40 or 45-50',
  'your ticket:\n7,1,14',
  'nearby tickets:\n7,3,47\n40,4,50\n55,2,20\n38,6,12'],
 'nearby tickets:\n7,3,47\n40,4,50\n55,2,20\n38,6,12',
 3)

In [4]:
inp = open('input\\16.txt').read()
inp = [line.rstrip('\n') for line in inp.split('\n\n')]
inp[:2], len(inp)

(['departure location: 40-261 or 279-955\ndeparture station: 33-375 or 394-963\ndeparture platform: 39-863 or 877-970\ndeparture track: 30-237 or 256-955\ndeparture date: 47-731 or 741-950\ndeparture time: 38-301 or 317-954\narrival location: 26-598 or 623-969\narrival station: 50-835 or 854-971\narrival platform: 44-535 or 549-958\narrival track: 36-672 or 685-967\nclass: 34-217 or 236-974\nduration: 29-469 or 483-970\nprice: 45-111 or 120-965\nroute: 32-751 or 760-954\nrow: 25-321 or 339-954\nseat: 38-423 or 438-958\ntrain: 45-798 or 813-954\ntype: 40-487 or 503-954\nwagon: 46-916 or 938-949\nzone: 25-160 or 184-957',
  'your ticket:\n73,59,83,127,137,151,71,139,67,53,89,79,61,109,131,103,149,97,107,101'],
 3)

In [5]:
def partA(l):
    valid_numbers = set()
    error_rate = 0
    for line in l[0].split('\n'):
        a,b,c,d = map(int, re.findall(r'\d+', line))
        valid_numbers |= set(range(a, b+1))
        valid_numbers |= set(range(c, d+1))
    for ticket in l[2].split('\n')[1:]:
        error_rate += sum(number for number in map(int, ticket.split(',')) if number not in valid_numbers)      
    
    return error_rate

partA(test_list)

71

In [6]:
partA(inp)

26869

# part B

In [24]:
testcase = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""

testcase = testcase.split('\n\n')
testcase

['class: 0-1 or 4-19\nrow: 0-5 or 8-19\nseat: 0-13 or 16-19',
 'your ticket:\n11,12,13',
 'nearby tickets:\n3,9,18\n15,1,5\n5,14,9']

In [138]:
def partB(l):
    valid_numbers = set()
    valid_tickets = set()
    fields = {}
    for line in l[0].split('\n'):
        a,b,c,d = map(int, re.findall(r'\d+', line))
        valid_numbers |= set(range(a, b+1))
        valid_numbers |= set(range(c, d+1))
        key = line.split(':')[0]
        fields[key] = set(range(a, b+1)).union(set(range(c, d+1)))
    
    myticket = l[1].split(':')[1]
    myticket = tuple(map(int, myticket.split(',')))
    #print('myticket', myticket)
    
    for ticket in l[2].split('\n')[1:]:
        ticket = tuple(map(int, ticket.split(',')))
        is_valid = all(number in valid_numbers for number in ticket)
        if is_valid:
            valid_tickets.add(ticket)
    
    all_rules = set(fields.keys())
    
    # create dict: key = index of each row, val = all possible fields
    lenticket = len(ticket)
    rowdict = {idx: all_rules.copy() for idx in range(lenticket)}
    #print(rowdict)
    
    # delete fields that do not match row (foreach ticket)
    for idx in range(lenticket):
        for rule in all_rules:
            for ticket in valid_tickets:
                if ticket[idx] not in fields[rule]:
                    #print(ticket[idx], 'not in ', rule)
                    rowdict[idx].remove(rule)
    
    
    # sort dict by number of rules. The first should have 1 rule, the 2nd 2 rules etc
    #  remove the rules that were already added to the solution (rules_seen)
    sol = [0] * lenticket
    rules_seen = set()
    for idx in sorted(rowdict, key=rowdict.get):
        rule = rowdict[idx] - rules_seen
        assert len(rule) == 1  # a single rule should be left
        #print(rule)
        rules_seen |= rule
        sol[idx] = rule.pop()
    
    print('solution =', sol)
      
    # calculate the answer  
    res = 1
    for idx, row in enumerate(sol):
        if row.startswith('departure'):
            print(idx, row)
            res *= myticket[idx] 
    
    return res 

partB(testcase)

solution = ['row', 'class', 'seat']


1

In [139]:
partB(inp)

solution = ['departure track', 'arrival track', 'wagon', 'arrival platform', 'departure location', 'departure date', 'row', 'seat', 'departure station', 'route', 'arrival station', 'departure time', 'type', 'train', 'zone', 'arrival location', 'duration', 'class', 'departure platform', 'price']
0 departure track
4 departure location
5 departure date
8 departure station
11 departure time
18 departure platform


855275529001